In [1]:
param_market = 'US'

In [2]:
import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

In [3]:
input_ = pd.read_excel('./item.xlsx',
                       header=0,
                       dtype=str).fillna('')

dict_item = {}
list_job = []
for i in range(len(input_)):
    if input_.loc[i, 'Vehicle_Page'] != '':
        vehicle_page = int(input_.loc[i, 'Vehicle_Page'])
        dict_item[input_.loc[i, 'Item_Number']] = {'vehicle_page': vehicle_page,
                                                   'crawler': 0,
                                                   'dict_vehicle': {},
                                                   'vehicle_1': '',
                                                   'vehicle_2': ''}
        for j in range(vehicle_page):
            list_job.append((i, j))

print('总数量：' + str(len(list_job)))
list_job

总数量：3214


[(6, 0),
 (6, 1),
 (41, 0),
 (41, 1),
 (238, 0),
 (239, 0),
 (239, 1),
 (240, 0),
 (241, 0),
 (244, 0),
 (247, 0),
 (247, 1),
 (247, 2),
 (250, 0),
 (250, 1),
 (251, 0),
 (251, 1),
 (252, 0),
 (252, 1),
 (252, 2),
 (253, 0),
 (253, 1),
 (255, 0),
 (255, 1),
 (255, 2),
 (256, 0),
 (257, 0),
 (257, 1),
 (260, 0),
 (263, 0),
 (264, 0),
 (264, 1),
 (265, 0),
 (265, 1),
 (265, 2),
 (268, 0),
 (269, 0),
 (270, 0),
 (270, 1),
 (271, 0),
 (275, 0),
 (275, 1),
 (275, 2),
 (277, 0),
 (277, 1),
 (278, 0),
 (278, 1),
 (278, 2),
 (279, 0),
 (280, 0),
 (280, 1),
 (280, 2),
 (280, 3),
 (280, 4),
 (280, 5),
 (280, 6),
 (283, 0),
 (283, 1),
 (286, 0),
 (286, 1),
 (287, 0),
 (287, 1),
 (287, 2),
 (288, 0),
 (289, 0),
 (290, 0),
 (290, 1),
 (290, 2),
 (292, 0),
 (292, 1),
 (293, 0),
 (300, 0),
 (300, 1),
 (304, 0),
 (306, 0),
 (306, 1),
 (307, 0),
 (307, 1),
 (307, 2),
 (310, 0),
 (310, 1),
 (312, 0),
 (312, 1),
 (312, 2),
 (312, 3),
 (312, 4),
 (312, 5),
 (312, 6),
 (312, 7),
 (312, 8),
 (312, 9),
 (312

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

ModuleNotFoundError: No module named 'UA'

In [ ]:
a = 0

i, j = list_job[a]

data = '{"scopedContext":{"catalogDetails":{"itemId":"' + input_.loc[i, 'Item_Number'] + '","categoryId":"' + input_.loc[i, 'Category_Id'] + '","marketplaceId":"' + input_.loc[i, 'Market_Id'] + '"}}}'

ua = UA.get_User_Agent_Requests()
ua['Content-Type'] = 'application/json'

if param_market == 'US':
    url = 'https://www.ebay.com/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&_ul=US&_stpos=91710&orig_cvip=true'
elif param_market == 'DE':
    url = 'https://www.ebay.de/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&_ul=DE&_stpos=10115&orig_cvip=true'
elif param_market == 'AU':
    url = 'https://www.ebay.com.au/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&ul=AU&_stpos=2019&orig_cvip=true'
elif param_market == 'UK':
    url = 'https://www.ebay.co.uk/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&_ul=GB&_stpos=BS81QU&orig_cvip=true'

resp = requests.post(url,
                     data=data,
                     headers=ua,
                     proxies=Proxy.get_Proxy_Requests()).text

resp

In [ ]:
import json

In [ ]:
json_data = json.loads(resp)

json_data

In [ ]:
list_header = json_data['modules']['COMPATIBILITY_TABLE']['paginatedTable']['header']['cells']

list_header

In [ ]:
if param_market in ['US', 'AU', 'UK']:
    name_make = 'Make'
    name_model = 'Model'
    name_engine = 'Engine'
    name_year = 'Year'
elif param_market == 'DE':
    name_make = 'Marke'
    name_model = 'Modell'
    name_engine = 'Motor'
    name_year = 'Baujahr'
    
for k in range(len(list_header)):
    if name_make in list_header[k]['textSpans'][0]['text']:
        col_make = k
    elif name_model in list_header[k]['textSpans'][0]['text']:
        col_model = k
    elif name_engine in list_header[k]['textSpans'][0]['text']:
        col_engine = k
    elif name_year in list_header[k]['textSpans'][0]['text']:
        col_year = k
        
col_make, col_model, col_engine, col_year

In [ ]:
list_row = json_data['modules']['COMPATIBILITY_TABLE']['paginatedTable']['rows']

list_row

In [ ]:
for row in list_row:
    make = row['cells'][col_make]['textSpans'][0]['text'].strip()
    model = row['cells'][col_model]['textSpans'][0]['text'].strip()
    engine = row['cells'][col_engine]['textSpans'][0]['text'].strip()
    if param_market in ['US', 'AU', 'UK']:
        list_year = [int(row['cells'][col_year]['textSpans'][0]['text'].strip())]
    elif param_market == 'DE':
        list_year = [int(year.split('/').strip()) for year in row['cells'][col_year]['textSpans'][0]['text'].split('-')]
        
    if make not in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle']:
        dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make] = {model: {engine: list_year}}
    else:
        if model not in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make]:
            dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model] = {engine: list_year}
        else:
            if engine not in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model]:
                dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine] = list_year
            else:
                dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine] += list_year
                
    dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine] = list(set(dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine]))
    dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine].sort()
    
dict_item[input_.loc[i, 'Item_Number']]['crawler'] += 1
    
dict_item[input_.loc[i, 'Item_Number']]

In [ ]:
dict_item[input_.loc[i, 'Item_Number']]['vehicle_page'] == dict_item[input_.loc[i, 'Item_Number']]['crawler']

In [ ]:
list_vehicle = []
for make in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle']:
    for model in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make]:
        list_year = []
        for engine in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model]:
            list_year += dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine]
        list_year = list(set(list_year))
        list_year.sort()
        
        year_min = list_year[0]
        year_max = list_year[-1]
        if year_min == year_max:
            list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
        else:
            list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

list_vehicle.sort()

dict_item[input_.loc[i, 'Item_Number']]['vehicle_1'] = '\n'.join(list_vehicle)

print(dict_item[input_.loc[i, 'Item_Number']]['vehicle_1'])

In [ ]:
list_vehicle = []
for make in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle']:
    for model in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make]:
        for engine in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model]:
            year_min = dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine][0]
            year_max = dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine][-1]
            if year_min == year_max:
                if engine == '':
                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                else:
                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + ' ' + engine)
            else:
                if engine == '':
                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))
                else:
                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' ' + engine)
                
list_vehicle.sort()

dict_item[input_.loc[i, 'Item_Number']]['vehicle_2'] = '\n'.join(list_vehicle)

print(dict_item[input_.loc[i, 'Item_Number']]['vehicle_2'])

In [ ]:
df_temp = pd.DataFrame([{'Item_Number': input_.loc[i, 'Item_Number'],
                         'Vehicle_1': dict_item[input_.loc[i, 'Item_Number']]['vehicle_1'],
                         'Vehicle_2': dict_item[input_.loc[i, 'Item_Number']]['vehicle_2']}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

len(output_correct)

In [ ]:
crawler_status = 'ok'

crawler_status

In [ ]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Item_Number': input_.loc[i, 'Item_Number'],
                             'Vehicle_Page': input_.loc[i, 'Vehicle_Page'],
                             'Category_Id': input_.loc[i, 'Category_Id'],
                             'Market_Id': input_.loc[i, 'Market_Id']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

len(output_error)

In [ ]:
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Item_Number'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./test_vehicle.xlsx', index=False)

output_correct

In [ ]:
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Item_Number'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./test_vehicle_error.xlsx', index=False)

output_error